# **Introduction**

In this activity, we will be updating our previous object detection model to YOLOv7 and integrating it with DeepSORT for optimization. The goal is to analyze the impact of the optimization algorithm on the model results and compare the performance of the updated models with the previous ones. We will be using the same dataset as before, but this time we will train the models using time frames ranging from 5 PM to 8 PM. We will be creating three versions of the model - **_small, medium, and large_ - and evaluating their performance using metrics such as F1 curve, P curve, mAP (mean average precision), accuracy, precision, and recall.**

To ensure a fair comparison, we will use the same dataset for each model and evaluate their performance separately. We will create a confusion matrix for each model and compare their accuracy, precision, and recall. We will also visualize the training progress of each model using time series graphs.

## Import Libraries

In [1]:
import torch                                    # For deep learning
import os                                       # For operating system related tasks 
import urllib.request                           # For opening and reading URLs
import glob                                     # For file handling
import shutil                                   # For file operations
from IPython.display import Image, clear_output # For displaying images and clearing the output

In [2]:
# Determine the device that PyTorch is using
device = torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'

# Print the setup information
print(f"Setup complete. Using torch version: {torch.__version__} on device: {device}")

Setup complete. Using torch version: 2.1.2+cu118 on device: NVIDIA GeForce RTX 3060 Laptop GPU


## Download YOLOv7 repository and install requirements


In [3]:
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -qr requirements.txt

c:\Users\Ryoji\Documents\Code\M1- SA (YOLOv7 + DeepSORT)\yolov7


Cloning into 'yolov7'...
c:\Users\Ryoji\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
# Get the weights file from the URL and save it to the current directory
url = "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt"
filename = "yolov7_training.pt"

urllib.request.urlretrieve(url, filename)

('yolov7_training.pt', <http.client.HTTPMessage at 0x1ecea4c0850>)

## Import Dataset (Robotflow)

Roboflow offers an API to export datasets, and in this case, the YOLOv8 format is used at this [URL](https://app.roboflow.com/data-science-3-5kybl/helmet-detection-lcyce/1).

![](https://i.imgur.com/ScfGIey.png)

In [6]:
!pip install roboflow                       

from roboflow import Roboflow                   # For downloading the dataset
rf = Roboflow(api_key="weuUhM15xraEmLMEAiPU")
project = rf.workspace("data-science-3-5kybl").project("helmet-detection-lcyce")
version = project.version(1)
dataset = version.download("yolov7")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Helmet-Detection-1 in yolov7pytorch:: 100%|██████████| 12971/12971 [00:23<00:00, 548.85it/s] 


## Train YOLOv7 model with Tune Hyperparameters

### A. Small Model

In [ ]:
# Execute the training script with the downloaded dataset and the weights file we downloaded earlier
!python train.py \
    --batch 64 \
    --epochs 50 \
    --img-size 640 \
    --data /yolov7/Helmet-Detection-1/data.yaml \
    --weights 'yolov7_training.pt' \
    --cache

In [ ]:
# Execute evaluation
!python detect.py \
    --weights runs/train/exp/weights/best.pt \
    --conf 0.1 \
    --source {dataset.location}/test/images

In [ ]:
# Set maximum images to print
limit = 10 

# Loop through each image in the directory
for i, imageName in enumerate(glob.glob('/content/yolov7/runs/detect/exp/*.jpg')):
    if i >= limit: # Stop if limit is reached
        break
    display(Image(filename=imageName)) # Display image
    print("\n")

#### A.1. TensorFlow

To monitor the training progress and visualize the logs, you can start TensorBoard and launch it after you have started training. The logs are saved in the `runs` folder.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Start TensorBoard within the notebook using magics
%tensorboard --logdir runs

In [ ]:
%cd ..

#### A.2. DeepSort

To detect and track objects in videos or live camera feeds combines with YOLOv7 model. This can be useful in various applications such as surveillance, autonomous vehicles, and object recognition.

In [ ]:
!git clone https://github.com/deshwalmahesh/yolov7-deepsort-tracking
%cd yolov7-deepsort-tracking

In [ ]:
# Copy the trained weights to the current directory and rename it
source_file = '/content/yolov7/runs/train/exp/weights/best.pt'
destination_file = '/content/yolov7x.pt'

shutil.copy(source_file, destination_file)


Ensure to modify the specified lines in the following Python files before executing this code:

- Modify line 247 in yolov7-deepsort/tracking_helpers.py to set `classes = ['Full_Face','Half_Face','Invalid','No_Helmet','Rider']`.
- Modify line 15 in yolov7-deepsort/data/coco.yml to set `classes = ['Full_Face','Half_Face','Invalid','No_Helmet','Rider']`, and; 
- Line 12 to set `nc: 5`.

After making these changes, you can proceed with running the code.

In [ ]:
from detection_helpers import *                 # For detection related functions
from tracking_helpers import *                  # For tracking related functions
from bridge_wrapper import *                    # For bridge related functions
from PIL import Image                           # For image processing 

In [ ]:
# Create a detector object
detector = Detector() 

# Set the path to the trained model and load it to the detector object
model_path = '/content/yolov7x.pt' 
detector.load_model(model_path)

# If the result is a 3D array, convert it to an image using OpenCV
if len(result.shape) == 3:  
    result = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

# Save the result image to a folder
result.save('__path__')

# Display the result image
result

In [ ]:
# Create a tracker object with the YOLOv7 model and the ReID model
tracker = YOLOv7_DeepSORT(reID_model_path="/content/yolov7/yolov7-deepsort-tracking/deep_sort/model_weights/mars-small128.pb", detector=detector)

In [ ]:
# Track the video and save the result to a file
tracker.track_video("__path__", output_folder="__path__", show_live=False, skip_frames=0, count_objects=True, verbose=2)


### B. Medium Model

In [ ]:
# Execute the training script again with the downloaded dataset but different weights file
!python train.py \
    --batch 32 \
    --epochs 75 \
    --img-size 640 \
    --data /yolov7/Helmet-Detection-1/data.yaml \
    --weights 'yolov7_training.pt' \
    --cache

In [ ]:
# Execute evaluation
!python detect.py \
    --weights runs/train/exp/weights/best.pt \
    --conf 0.1 \
    --source {dataset.location}/test/images

In [ ]:
# Set maximum images to print
limit = 10 

# Loop through each image in the directory
for i, imageName in enumerate(glob.glob('/content/yolov7/runs/detect/exp/*.jpg')):
    if i >= limit: # Stop if limit is reached
        break
    display(Image(filename=imageName)) # Display image
    print("\n")

#### B.1. TensorFlow

To monitor the training progress and visualize the logs, you can start TensorBoard and launch it after you have started training. The logs are saved in the `runs` folder.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Start TensorBoard within the notebook using magics
%tensorboard --logdir runs

In [ ]:
%cd ..

#### B.2. DeepSort

To detect and track objects in videos or live camera feeds combines with YOLOv7 model. This can be useful in various applications such as surveillance, autonomous vehicles, and object recognition.

In [ ]:
# Copy the trained weights to the current directory and rename it
source_file = '/content/yolov7/runs/train/exp/weights/best.pt'
destination_file = '/content/yolov7x.pt'

shutil.copy(source_file, destination_file)